In [ ]:
import networkx as nx
from random import choice, choices
import gensim
from math import sqrt
from operator import itemgetter
import numpy as np

def readData(filename):
    data = []
    with open(filename) as f:
        for line in f:
         data.append(line.strip().split())
    return data

def evaluation(topList, groundTruth):
    count = 0
    for link in topList:
        if link[0] in groundTruth:
            count += 1
    print(float(count) / len(groundTruth) * 100, '%')

def buildGraph(data):
    G = nx.Graph()
    for pair in data:
        G.add_edge(pair[0],pair[1])
    return G

def get_bias(b_centralities, neighbors):
    bias = []
    for node in neighbors:
        bias.append(b_centralities[node])
    return bias

def randomWalk(G,length,strategy, B):
    if strategy == 'bfs&dfs':
        walks = []
        #length = 4
        print('Generating dfs&bfs mixed random walks...')
        for i in range(20):
            #repeat 20 times to collect more walks for each node
            for node in G.nodes:
                #start the random walk with each node as the starter
                walk = [node]
                walkLength = 0
                while walkLength<length:
                    node = choice(list(G.neighbors(node)))
                    walk.append(node)
                    node = list(G.neighbors(node))[0]
                    walk.append(node)
                    walkLength += 1
                walks.append(walk)
        return walks
    elif strategy == 'bias':
        print('Generating betweeness centralities...')
        #length = 23
        b_centralities = B
    walks = []
    print('Generating random walks...')
    for i in range(5):
        #repeat 5 times to collect more walks for each node
        for node in G.nodes:
            #start the random walk with each node as the starter
            walk = [node]
            walkLength = 0
            while walkLength<length:
                if strategy == 'random':
                    node = choice(list(G.neighbors(node)))
                elif strategy == 'bias':
                    neighbors = list(G.neighbors(node))
                    biases = get_bias(b_centralities, neighbors)
                    node = choices(neighbors, biases)[0]
                walk.append(node)
                walkLength += 1
            walks.append(walk)
    return walks

def skipGram(walks, method):
    print('Training Skip-Gram model... Pleas wait.')
    #using Skip-Gram
    if method == 'bfs&dfs':
        model = gensim.models.Word2Vec(walks,window=8,sg=1)
    elif method == 'bias':
        model = gensim.models.Word2Vec(walks,window=8,sg=4)
    elif method == 'random':
        model = gensim.models.Word2Vec(walks,window=5,sg=1)
    return model

def computeProximityScore(em1,em2):
    #Cosine Similarity
    score = np.dot(em1,em2)/(np.linalg.norm(em1)*np.linalg.norm(em2))
    #Euclidean distance
    #score = 1/(em1-em2).dot(em1-em2)
    return score

In [ ]:
#read data from files
trainingData = readData('training.txt')
valid_pos = readData('val_positive.txt')
valid_neg = readData('val_negative.txt')
testData = readData('test.txt')
validationData = valid_pos+valid_neg
#build graph
G = nx.read_edgelist('training.txt')

In [ ]:
B = nx.betweenness_centrality(G)

In [ ]:
#generate random walks
walks = randomWalk(G,10,'bfs&dfs', B)
#train model
model = skipGram(walks, 'bfs&dfs')
#compute proximity score
linkScores={}
for pair in validationData:
    linkScores[pair[0]+' '+pair[1]] = computeProximityScore(model.wv[pair[0]],model.wv[pair[1]])
top100Links = sorted(linkScores.items(), key=itemgetter(1))[::-1][:100]
#validation & evaluation
groundTruth={}.fromkeys([pair[0]+' '+pair[1] for pair in valid_pos])
evaluation(top100Links,groundTruth)

In [ ]:
def adar_score(author1, author2, neighbors):
    return sum([1.0/math.log(len(neighbors[v])) for v in neighbors[author1].intersection(neighbors[author2])])

def getNeighbors(data):
    neighbors = defaultdict(set)
    for pair in data:
        neighbors[pair[0]].add(pair[1])
        neighbors[pair[1]].add(pair[0])
    return neighbors

neighbors = getNeighbors(trainingData)
val_list = []
for i in range(len(val_pos[0])):
    val_list.append(tuple(val_pos[:,i]))

In [ ]:
Ascore = {}
for i in range(len(val_list)):
    Ascore[val_list[i]] = adar_score(val_list[i][0], val_list[i][1], 'adar', neighbors)
top_list = []
for i in range(100):
    max_score = max(Ascore, key=Ascore.get)
    top_list.append(max_score)
    Ascore.pop(max_score)
evaluation(top_list, val_pos_list)

In [ ]:
from igraph import *
from math import *
import sys
import numpy as np
import os
import glob
import random
from random import shuffle
from random import seed
import matplotlib.pyplot as plt
import time
import datetime
import collections

maxl = 2
def katz_similarity(katzDict,i,j):
	l = 1
	neighbors = katzDict[i]
	score = 0

	while l <= maxl:
		numberOfPaths = neighbors.count(j)
		if numberOfPaths > 0:
			score += (beta**l)*numberOfPaths

		neighborsForNextLoop = []
		for k in neighbors:
			neighborsForNextLoop += katzDict[k]
		neighbors = neighborsForNextLoop
		l += 1

	return score

def get_edge_list(dataset_path):
	data_file = open(dataset_path)
	edge_list = map(lambda x:tuple(map(int,x.split())),data_file.read().split("\n")[:-1])
	data_file.close()
	return edge_list

# Convert edge_list into a set of constituent edges
def get_vertices_set(edge_list):
	res = set()
	for x,y in edge_list:
		res.add(x)
		res.add(y)
	return res

# Calculates accuracy metrics (Precision & Recall),
# for a given similarity-model against a test-graph.
def print_precision_and_recall(sim,train_graph,test_graph,test_vertices_set,train_vertices_set):
	precision = recall = c = 0
	for i in test_vertices_set:
		if i in train_vertices_set:
			actual_friends_of_i = set(test_graph.neighbors(i))

			# Handles case where test-data < k
			if len(actual_friends_of_i) < k:
				k2 = len(actual_friends_of_i)
			else:
				k2 = k

			top_k = set(get_top_k_recommendations(train_graph,sim,i,k2))

			precision += len(top_k.intersection(actual_friends_of_i))/float(k2)
			recall += len(top_k.intersection(actual_friends_of_i))/float(len(actual_friends_of_i))
			c += 1
	print('Precision is : ' + str(precision/c))
	print('Recall is : ' + str(recall/c))

In [ ]:
def KatzScore(u, v, G):
    a = 1
    neighbors = list(G.neighbors(u))
    score = 0
    while a <= 2:
        numberofPath = neighbors.count(v)
        if numberofPath > 0:
            score += (0.05**a)*numberofPath
        neighborsForNextLoop = []
        for k in neighbors:
            neighborsForNextLoop += list(G.neighbors(k))
        neighbors = neighborsForNextLoop
        a += 1
    return score

#read data from files
trainingData = readData('training.txt')
valid_pos = readData('val_positive.txt')
valid_neg = readData('val_negative.txt')
testData = readData('test.txt')
validationData = valid_pos+valid_neg
# Build graph
train_graph = nx.read_edgelist('training.txt')

# Katz score
linkScores={}
for pair in validationData:
    linkScores[pair[0]+' '+pair[1]] = KatzScore(pair[0], pair[1], train_graph)
top100Links = sorted(linkScores.items(), key=itemgetter(1))[::-1][:100]
#validation & evaluation
groundTruth={}.fromkeys([pair[0]+' '+pair[1] for pair in valid_pos])
evaluation(top100Links,groundTruth)

In [7]:
import networkx as nx
def KatzScore(u, v, G):
    a = 1
    neighbors = list(G.neighbors(u))
    score = 0
    while a <= 1:
        numberofPath = neighbors.count(v)
        if numberofPath > 0:
            score += (0.05**a)*numberofPath
        neighborsForNextLoop = []
        for k in neighbors:
            neighborsForNextLoop += list(G.neighbors(k))
        neighbors = neighborsForNextLoop
        a += 1
    return score

train = nx.read_edgelist('training.txt')
valid_pos = nx.read_edgelist('val_positive.txt')
valid_neg = nx.read_edgelist('val_negative.txt')
test = nx.read_edgelist('test.txt')

valid_all =[pair for pair in valid_pos.edges()] + [pair for pair in valid_neg.edges()] 

Katz = {}
for pair in Valid_all:
    Katz[pair[0] + ' ' + pair[1]] = KatzScore(pair[0], pair[1], train)
    
groundTruth={}.fromkeys([pair[0]+' '+pair[1] for pair in valid_pos.edges()])

top100Links = sorted(Katz.items(), key = lambda d:d[1], reverse = True)[:100]
count = 0
for link in top100Links:
    if link[0] in groundTruth:
        count += 1
count/len(groundTruth)

0.85

In [ ]:
train_list = get_edge_list('training.txt')
test_list = get_edge_list('val_positive.txt')
train_graph = Graph(train_list)
test_graph = Graph(test_list)
train_n = train_graph.vcount()
#train_vertices_set = get_vertices_set(train_list) # Need this because we have to only consider target users who are present in this train_vertices_set
#test_vertices_set = get_vertices_set(test_list) # Set of target users
train_vertices_set = nx.read_edgelist('training.txt').nodes()
test_vertices_set = nx.read_edgelist('valid_neg.txt').nodes()


# build a special dict that is like an adjacency list
katzDict = {}
adjList = train_graph.get_adjlist()

for i, l in enumerate(adjList):
    katzDict[i] = l

sim = [[0 for i in range(train_n)] for j in range(train_n)]
for i in range(train_n):
    #print(i)
    if i not in train_vertices_set:
        continue

    for j in range(i+1, train_n):
        if j in train_vertices_set:		# TODO: check if we need this
            sim[i][j] = sim[j][i] = katz_similarity(katzDict,i,j)

print_precision_and_recall(sim,train_graph,test_graph,test_vertices_set,train_vertices_set)

In [ ]:
a = {1,2}
a.update({1,4})
a

In [ ]:
T = nx.dfs_tree(G, '339')
list(T.nodes())[:20]

In [ ]:
dfs = []
for edge in list(nx.dfs_edges(G, '339')):
    dfs.append(edge[1])
dfs[:20]

In [ ]:
list(G.neighbors('339'))

In [ ]:
list(iter(G['339']))

In [ ]:
bfs_edges_(G, '8193')

In [ ]:
def bfs_edges_(G, source, reverse=False):
    if reverse and isinstance(G, nx.DiGraph):
        neighbors = G.predecessors_iter
    else:
        neighbors = G.neighbors_iter
    print(neighbors)
    visited = set([source])
    queue = deque([(source, neighbors(source))])
    while queue:
        parent, children = queue[0]
        print(children)
        try:
            child = next(children)
            if child not in visited:
                yield parent, child
                visited.add(child)
                queue.append((child, neighbors(child)))
        except StopIteration:
            queue.popleft()

In [ ]:
source = '339'
reverse = None
if reverse and isinstance(G, nx.DiGraph):
    neighbors = G.predecessors_iter
else:
    neighbors = G.neighbors
visited = set([source])
list(neighbors(source))

In [ ]:
T = nx.bfs_tree(G, '339')
list(T.nodes())